In [1]:
import pandas as pd
import numpy as np

# Importing Dataset

In [3]:
df = pd.read_csv('/content/drive/MyDrive/Datasets/fashion-mnist_train.csv')

# Splitting and Scalling

In [9]:
from sklearn.model_selection import train_test_split

X = df.drop('label', axis=1)
y = df['label']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

train_x = X_train/255.0
test_x = X_test/255.0

# Check for GPU Availability

In [8]:
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

if device.type == 'cuda':
    print(torch.cuda.get_device_name(0))

Tesla T4


# Dataset Class and DataLoader

In [11]:
from torch.utils.data import Dataset, DataLoader

class CustomDataSet(Dataset):
  def __init__(self, x, y):
    self.x = torch.tensor(x.values, dtype=torch.float32)
    self.y = torch.tensor(y.values, dtype=torch.long)

  def __getitem__(self, i):
    return self.x[i], self.y[i]

  def __len__(self):
    return len(self.x)

train_data = CustomDataSet(train_x, y_train)
test_data = CustomDataSet(test_x, y_test)

train_loader = DataLoader(train_data, batch_size=64, shuffle=True, pin_memory=True)
test_loader = DataLoader(test_data, batch_size=64, shuffle=False, pin_memory=True)

# Defining the Model

In [14]:
import torch.nn as nn

class MyModel(nn.Module):

  def __init__(self, num_features):
    super().__init__()

    self.model = nn.Sequential(
        nn.Linear(num_features, 128),
        nn.ReLU(),
        nn.Linear(128, 64),
        nn.ReLU(),
        nn.Linear(64, 10)
    )

  def forward(self, x):
    return self.model(x)

In [13]:
# Defining Parameter
learning_rate = 0.1
epochs = 100

In [15]:
# Initialising Model
model = MyModel(train_x.shape[1]).to(device)

# loss Function
loss_fn = nn.CrossEntropyLoss()

# Optimizer
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

# Model Pipeline

In [16]:
for epoch in range(epochs):

  total_epoch_loss = 0
  for batch_features, batch_labels in train_loader:
    batch_features = batch_features.to(device)
    batch_labels = batch_labels.to(device)

    # Forward Pass
    output = model(batch_features)
    loss = loss_fn(output, batch_labels)

    # Backward Pass
    optimizer.zero_grad()
    loss.backward()

    # Update Gradient
    optimizer.step()

    total_epoch_loss += loss.item()

  print(f'Epoch: {epoch+1}, Loss: {total_epoch_loss/len(train_loader)}')

Epoch: 1, Loss: 0.7393190166950225
Epoch: 2, Loss: 0.4731459547877312
Epoch: 3, Loss: 0.4175094682971636
Epoch: 4, Loss: 0.3852225206295649
Epoch: 5, Loss: 0.3607098419268926
Epoch: 6, Loss: 0.34249720214804014
Epoch: 7, Loss: 0.32889119786024096
Epoch: 8, Loss: 0.31813872306545576
Epoch: 9, Loss: 0.3053435844282309
Epoch: 10, Loss: 0.2966456120312214
Epoch: 11, Loss: 0.2848380032082399
Epoch: 12, Loss: 0.27701505487660566
Epoch: 13, Loss: 0.2684634185930093
Epoch: 14, Loss: 0.26194801773130894
Epoch: 15, Loss: 0.2551481956243515
Epoch: 16, Loss: 0.24858039044837157
Epoch: 17, Loss: 0.24080341357986132
Epoch: 18, Loss: 0.23550920505325
Epoch: 19, Loss: 0.230264292349418
Epoch: 20, Loss: 0.22265910304586092
Epoch: 21, Loss: 0.21982631140947342
Epoch: 22, Loss: 0.21542733469605446
Epoch: 23, Loss: 0.2086347139775753
Epoch: 24, Loss: 0.20537772222856682
Epoch: 25, Loss: 0.2007841658691565
Epoch: 26, Loss: 0.19846336142718793
Epoch: 27, Loss: 0.19381008500854174
Epoch: 28, Loss: 0.18827655

# Model Evaluation

In [17]:
model.eval()

total_correct = 0
total_samples = 0

with torch.no_grad():
  for batch_features, batch_labels in test_loader:
    batch_features, batch_labels = batch_features.to(device), batch_labels.to(device)

    output = model(batch_features)
    _, predicted = torch.max(output, 1)

    total_samples += batch_labels.size(0)
    total_correct += (predicted == batch_labels).sum().item()

accuracy = total_correct / total_samples
print(f'Accuracy: {accuracy}')

Accuracy: 0.8874166666666666


In [18]:
model.eval()

total_correct = 0
total_samples = 0

with torch.no_grad():
  for batch_features, batch_labels in train_loader:
    batch_features, batch_labels = batch_features.to(device), batch_labels.to(device)

    output = model(batch_features)
    _, predicted = torch.max(output, 1)

    total_samples += batch_labels.size(0)
    total_correct += (predicted == batch_labels).sum().item()

accuracy = total_correct / total_samples
print(f'Accuracy: {accuracy}')

Accuracy: 0.9750416666666667
